# Simple 10-class classification

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Suppress warkings (gets rid of some type-conversion warnings)
warnings.filterwarnings("ignore")
%matplotlib inline

### Generate some dummy data

In [ ]:
classes = 10
data = np.zeros((1000, 100))
labels = np.zeros((1000, 1), dtype=int)

poles = np.round(np.concatenate(
    (np.random.uniform(0,classes,(classes,2)),
    np.random.uniform(-5,5,(classes,4))),axis=1))

#Using the poles, generate 100 images for each of them.
points = 5000
fig=plt.figure(figsize=(15, 7))
for i in range(0, len(poles)):
    p = poles[i,0:2]
    for j in range(0,100):
        sigma = poles[i, 2:6].reshape(2,2) + np.random.uniform(-0.5,0.5,(2,2))
        x,y = np.random.multivariate_normal(p,sigma,points).T
        H = np.histogram2d(x,y,bins=classes, range=[[0,classes],[0,classes]], normed=True)[0].T
        data[i*100+j] = H.reshape(1,100)
        labels[i*100+j] = i

### (Optional) Visualization of the data
This is not part of the Keras example, but it helps to understand what we are trying to do.

In [ ]:
# Plot a 2D representation of the data, using t-SNE
from sklearn.manifold import TSNE
data_viz = TSNE(n_components=2).fit_transform(data)
print("Data dimensions after reduction: {}".format(data_viz.shape))

In [ ]:
plt.scatter(data_viz[:,0], data_viz[:,1], c=labels[:,0], cmap=plt.cm.get_cmap("jet", classes))
plt.colorbar(ticks=range(classes))

#### Let's see what each example looks like
We can think of them as the images of "digits." We will actually train character recognition in future tutorials.

In [ ]:
sampleSize = 10
samples = np.array(range(0,1000,100)) + np.random.randint(0,100,(1,classes))
samples = samples[0,:].tolist()

fig=plt.figure(figsize=(15, 8))
for i in range(0, sampleSize):
    fig.add_subplot(2, 5, i+1, aspect='equal')
    plt.imshow(np.reshape(data[samples[i],:], (10,10)), interpolation='nearest', cmap="hot", origin='low')
    plt.title('Class {}\n({},{})'.format(labels[samples[i]], 
                                        poles[labels[samples[i]], 0].astype(int), 
                                        poles[labels[samples[i]], 1].astype(int)))
    plt.xlabel("Img {}".format(samples[i]))

## Finally, let's use Keras

### Create the model

In [ ]:
# For a single-input model with 10 classes (categorical classification):
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(classes, activation='softmax'))

### Compile the model

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Transform labels (i.e., the outputs), to the shape expected by the model

In [ ]:
# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=classes)

# Optional: visualize the label transformation
rIdx = np.random.randint(0, labels.shape[0])
print("Label shapes before: {}".format(labels.shape))
print("\tLabel at random index {}:\n\t{}\n".format(rIdx, labels[rIdx]))

print("Label shapes after: {}".format(one_hot_labels.shape))
print("\tOne-hot encoded label at random index {} (same as above):\n\t{}\n".format(rIdx, one_hot_labels[rIdx, :]))
print("(Pos.)\t{}".format(np.array(range(0,10),dtype="float")))

### Train the model
Note how the loss decreases, while the accuracy increases, as the training goes through more and more epochs.

In [ ]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=100, batch_size=32, shuffle=True, verbose=1)

In [ ]:
predSetSize = 10
predData = np.zeros((predSetSize, 100))
samples = np.random.permutation(poles.shape[0])[0:predSetSize].tolist()

for i in range(0, len(samples)):
    p = poles[samples[i],0:2]
    sigma = poles[samples[i], 2:6].reshape(2,2) + np.random.uniform(-0.5,0.5,(2,2))
    x,y = np.random.multivariate_normal(p,sigma,points).T
    H = np.histogram2d(x,y,bins=classes, range=[[0,classes],[0,classes]], normed=True)[0].T
    predData[i] = H.reshape(1,100)

results = np.round(model.predict(predData, verbose=1), decimals=2)
resultLabels = np.argmax(results, axis=1)

fig=plt.figure(figsize=(15, 8))
for i in range(0, predSetSize):
    fig.add_subplot(2,5, i+1)
    plt.imshow(np.reshape(predData[i], (10,10)), interpolation='nearest', cmap="hot", origin='low')
    plt.title('Class {}\n({},{})'.format(resultLabels[i], poles[resultLabels[i],0].astype(int), poles[resultLabels[i],1].astype(int)))
    plt.xlabel("Img {}".format(i))



## Conclusions
This example is still abstract (i.e., we used random data), but it shows the general workflow. In the next tutorial, we will apply this to a meaningful dataset.